In [13]:
import numpy as np
from typing import List, Tuple
from ultralytics import YOLO
from dataclasses import dataclass

from deep_sort.application_util import preprocessing
from deep_sort.deep_sort import nn_matching
from deep_sort.deep_sort.detection import Detection
from deep_sort.deep_sort.tracker import Tracker
from deep_sort.tools import generate_detections as gdet


# own
from utils.utils import (get_frames, SEQ_01, SEQ_02, SEQ_03)
from utils.deepsort_utils import LABELS_DICT, UNKNOWN_DEFAULT

In [2]:
deepsort_model_ = './networks/mars-small128.pb' # TODO missing
yolo_model_ = '../../models/yolov8n.pt' # TODO missing

encoder = gdet.create_box_encoder(deepsort_model_, batch_size=1)
_metric = nn_matching.NearestNeighborDistanceMetric("cosine", 0.4, None)
tracker = Tracker(_metric, n_init=0)
detector = YOLO(yolo_model_)

2023-05-05 17:55:27.735165: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [17]:
@dataclass
class DeepSortObject():
    id: int
    label: str
    confidence: float
    xyxy: List[float]
    mask: np.ndarray

    @property
    def xywh(self):
        raise NotImplementedError

In [20]:
frames = [(_l, _r) for _l, _r in [get_frames(frame_num_=i, seq_dir_=SEQ_01) for i in range(2)]]

detection_results = []
for _i_frame, _frame in enumerate(frames):
    _detections_i = detector(_frame, classes=[0, 1, 2])

    # boxes = _detections_i[0].boxes
    if not _detections_i: print(f"no detections in frame {_i_frame}");continue

    # detection_results.append(_detections_i) # QUESTION: Need to keep track of ALL detecitons?

    objects_i = []
    for _det in _detections_i[0].boxes: # 0: first (only) image | .boxes: list of detections
        _dso = DeepSortObject(
            id=int(_det.cls[0]),
            label=LABELS_DICT.get(int(_dso.id), UNKNOWN_DEFAULT),
            confidence=float(_det.conf[0]),
            xyxy=_det.xyxy.tolist()[0],
            mask=None # TODO get mask from detection
        )
        
        objects_i.append(_dso)
        


0: 256x640 3 persons, 2 bicycles, 1: 256x640 3 persons, 2 bicycles, 59.8ms
Speed: 0.4ms preprocess, 29.9ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 256x640 3 persons, 3 bicycles, 1: 256x640 2 persons, 2 bicycles, 58.4ms
Speed: 0.2ms preprocess, 29.2ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)


In [21]:
print(objects_i[0])

DeepSortObject(id=0, label='Pedestrian', confidence=0.8350500464439392, xyxy=[422.4808654785156, 202.17820739746094, 553.8821411132812, 467.69561767578125], mask=None)


In [6]:
dets_i = detection_results[0]

# _det_0 = _dets_i[0]

# print(f"confidences: {_det_0.conf[0]}")
# print(f"classes: {LABELS_DICT.get(int(_det_0.cls[0]))} ({_det_0.cls[0]})")



In [10]:
_dets_i = detection_results[0]

objects_i = []
for _det in _dets_i:
    _dso = DeepSortObject()
    _dso.id = int(_det.cls[0])
    _dso.label = LABELS_DICT.get(int(_dso.id), UNKNOWN_DEFAULT)
    _dso.confidence = float(_det.conf[0])
    _dso.xyxy = _det.xyxy.tolist()[0]
    _dso.mask = None
    
    objects_i.append(_dso)

5
